In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("tumor-data-without-healthy.xlsx")

In [3]:
labels = [value for column, value in data.iloc[0].items()]
data = data.set_axis(labels, axis=1)
data = data[data.columns[1:]].apply(
    lambda x: pd.to_numeric(x.replace('<', '').replace(',', '.'), errors='coerce')
)
data.drop(index=0, inplace=True)
data["Гормональная активность 0-нет               1-да"] = data["Гормональная активность 0-нет               1-да"].astype(str)
data["Гормональная активность 0-нет               1-да"] = data["Гормональная активность 0-нет               1-да"].fillna("missing")

In [4]:
""" dropping some columns """

data.dropna(axis=1, how="all", inplace=True)
data.dropna(axis=0, thresh=int(data.shape[1]*0.6), inplace=True)
data.dropna(axis=1, thresh=int(data.shape[0]*0.6), inplace=True)
data.drop(["Пол:    0-жен, 1-муж"], inplace=True, axis=1)

In [5]:
X = data.drop(columns=["КАН", "АКР"])
y = np.argmax(data[["КАН", "АКР"]], axis=1)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

In [7]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(loss_function = "Logloss",
                           cat_features = ["Гормональная активность 0-нет               1-да"],
                           verbose=5,
                           thread_count=-1)

In [8]:
"""from sklearn.model_selection import GridSearchCV

param_grid = {
    'iterations': [20, 40, 60],
    'learning_rate': [0.01, 0.1, 0.5],
    'depth': [6, 8, 10],
    'l2_leaf_reg': [1, 3, 5]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)"""

"from sklearn.model_selection import GridSearchCV\n\nparam_grid = {\n    'iterations': [20, 40, 60],\n    'learning_rate': [0.01, 0.1, 0.5],\n    'depth': [6, 8, 10],\n    'l2_leaf_reg': [1, 3, 5]\n}\n\ngrid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)\n\ngrid_search.fit(X_train, y_train)\nprint(grid_search.best_params_)"

In [14]:
best_model = CatBoostClassifier(iterations=20,
                                depth=10,
                                l2_leaf_reg=3,
                                learning_rate=0.1,
                                loss_function = "Logloss",
                                verbose=5,
                                thread_count=-1)

In [10]:
"""start_idx = 0
end_idx = int(len(X)*0.8)
step = 2
while end_idx <= X.shape[0]:
    current_training_data = (X[start_idx:end_idx], y[start_idx:end_idx])
    if start_idx == 0:
        current_validating_data = (X[end_idx:], y[end_idx:])
    else:
        current_validating_data = (np.concatenate((X[end_idx:], X[:start_idx])),  np.concatenate((y[end_idx:], y[:start_idx])))
    if start_idx == 0 :
        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True)
    else:
        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True,
                       init_model=best_model)
    start_idx += step
    end_idx += step"""

'start_idx = 0\nend_idx = int(len(X)*0.8)\nstep = 2\nwhile end_idx <= X.shape[0]:\n    current_training_data = (X[start_idx:end_idx], y[start_idx:end_idx])\n    if start_idx == 0:\n        current_validating_data = (X[end_idx:], y[end_idx:])\n    else:\n        current_validating_data = (np.concatenate((X[end_idx:], X[:start_idx])),  np.concatenate((y[end_idx:], y[:start_idx])))\n    if start_idx == 0 :\n        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True)\n    else:\n        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True,\n                       init_model=best_model)\n    start_idx += step\n    end_idx += step'

In [15]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    print(f"Fold {i}:")
    xtrain, xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    if i == 0:
        best_model.fit(xtrain, ytrain, eval_set=(xtest, ytest), plot=True)
    else:
        best_model.fit(xtrain, ytrain, eval_set=(xtest, ytest), init_model=best_model, plot=True)

Fold 0:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6255092	test: 0.6733228	best: 0.6733228 (0)	total: 13.8ms	remaining: 263ms
5:	learn: 0.3605764	test: 0.4844073	best: 0.4844073 (5)	total: 59.3ms	remaining: 138ms
10:	learn: 0.2534734	test: 0.4325968	best: 0.4325968 (10)	total: 116ms	remaining: 95.2ms
15:	learn: 0.1759529	test: 0.3869786	best: 0.3869786 (15)	total: 180ms	remaining: 45ms
19:	learn: 0.1299029	test: 0.3233390	best: 0.3233390 (19)	total: 209ms	remaining: 0us

bestTest = 0.3233390364
bestIteration = 19

Fold 1:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.1699134	test: 0.1268378	best: 0.1268378 (0)	total: 11.2ms	remaining: 213ms
5:	learn: 0.1297873	test: 0.1161036	best: 0.1161036 (5)	total: 61ms	remaining: 142ms
10:	learn: 0.1037776	test: 0.1113384	best: 0.1113384 (10)	total: 111ms	remaining: 90.9ms
15:	learn: 0.0852936	test: 0.1052391	best: 0.1052391 (15)	total: 152ms	remaining: 38ms
19:	learn: 0.0747153	test: 0.1036375	best: 0.1036375 (19)	total: 195ms	remaining: 0us

bestTest = 0.1036374785
bestIteration = 19

Fold 2:


...

0:	learn: 0.0821512	test: 0.0617478	best: 0.0617478 (0)	total: 10.1ms	remaining: 192ms
5:	learn: 0.0703217	test: 0.0571484	best: 0.0571484 (5)	total: 49.3ms	remaining: 115ms
10:	learn: 0.0599073	test: 0.0537117	best: 0.0537117 (10)	total: 88.5ms	remaining: 72.4ms
15:	learn: 0.0511020	test: 0.0501905	best: 0.0501905 (15)	total: 132ms	remaining: 33.1ms
19:	learn: 0.0465208	test: 0.0491148	best: 0.0491148 (19)	total: 174ms	remaining: 0us

bestTest = 0.04911478707
bestIteration = 19

Fold 3:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0480439	test: 0.0391918	best: 0.0391918 (0)	total: 9.56ms	remaining: 182ms
5:	learn: 0.0432454	test: 0.0385988	best: 0.0385988 (5)	total: 61.9ms	remaining: 144ms
10:	learn: 0.0391242	test: 0.0383335	best: 0.0383335 (10)	total: 109ms	remaining: 88.8ms
15:	learn: 0.0357115	test: 0.0381091	best: 0.0381091 (15)	total: 153ms	remaining: 38.3ms
19:	learn: 0.0332354	test: 0.0377223	best: 0.0375750 (17)	total: 193ms	remaining: 0us

bestTest = 0.03757501655
bestIteration = 17

Shrink model to first 18 iterations.
Fold 4:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0360786	test: 0.0277129	best: 0.0277129 (0)	total: 10.2ms	remaining: 195ms
5:	learn: 0.0330000	test: 0.0270366	best: 0.0270366 (5)	total: 61.5ms	remaining: 144ms
10:	learn: 0.0293838	test: 0.0247587	best: 0.0247587 (10)	total: 87.8ms	remaining: 71.8ms
15:	learn: 0.0272108	test: 0.0239442	best: 0.0239442 (15)	total: 137ms	remaining: 34.2ms
19:	learn: 0.0254948	test: 0.0234402	best: 0.0234402 (19)	total: 171ms	remaining: 0us

bestTest = 0.02344023861
bestIteration = 19



In [16]:
from sklearn.model_selection import cross_val_score

cv = cross_val_score(best_model, X, y, n_jobs=-1)
cv

array([0.95238095, 0.9047619 , 0.9       , 0.85      , 0.9       ])

In [17]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.86      0.92         7

    accuracy                           0.91        11
   macro avg       0.90      0.93      0.91        11
weighted avg       0.93      0.91      0.91        11



In [18]:
feature_importances = best_model.get_feature_importance()
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

importance_df

,Feature,Importance
3,Weiss,26.672042
1,"Максимальный размер, мм",14.815787
28,16dP2_3A,7.656574
12,Индекс свободный кортизон/свободный кортизол в...,4.182032
48,THF_THE1,2.502675
49,THF_THE,2.409132
37,alloTHF,2.328603
14,Et,2.259443
4,Кортизол крови утро,2.228118
51,aTHB_THB,2.189375
